In [ ]:
!pip install dash_leaflet
!pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.1/133.1 KB 7.3 MB/s eta 0:00:00
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 2.0.0
    Not uninstalling markupsafe at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'MarkupSafe'. No files were found to uninstall.
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Not uninstalling jinja2 at /shared-libs/python3.9/py-core/lib/python3.9/site-packages, outside environment /root/venv
    Can't uninstall 'Jinja2'. No files were found to uninstall.
ERROR: pip's dependency resolver do

In [ ]:
import requests
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import datetime
import pytz
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import folium
import dash_leaflet as dl

# Charger le dataset avec les parkings INDIGO et TM

In [ ]:
# Récupérer les données des parkings indigo et TM via un fichier csv
link_csv_parking_indigo = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/parcs-de-stationnement/exports/csv?lang=fr&timezone=Europe%2FBerlin&use_labels=true&delimiter=%3B"
df_parking_indigo = pd.read_csv(link_csv_parking_indigo,sep=";")

# Garder uniquement certaines colonnes
colonne_a_garder = ['xlong', 'ylat', 'nom', 'nb_places', 'adresse', 'type_ouvrage', 'gestionnaire', 'gratuit', 'nb_voitures', 'nb_velo']
df_parking_indigo_nettoye = df_parking_indigo.loc[:, colonne_a_garder]

# Remplacer les données de la colonne gratuit
df_parking_indigo_nettoye['gratuit'] = df_parking_indigo_nettoye['gratuit'].replace({'F': 'non', 'T': 'oui'})

# Concaténer la latitude avec la longitude
df_parking_indigo_nettoye['lat&lon'] = df_parking_indigo_nettoye.apply(lambda x: f"{x['ylat']}, {x['xlong']}", axis=1)

# Rajout de la colonne "Relais ?"
df_parking_indigo_nettoye['relais ?'] = 'non'


# Charger le dataset avec les parkings relais

In [ ]:
# Se connecter à l'API et charger les éléments dans un DataFrame
link_parking_relais = "https://data.toulouse-metropole.fr/api/records/1.0/search/?dataset=parkings-relais&q=&facet=commune"
response = requests.get(link_parking_relais)
data = response.json()
df_parking_relais = pd.DataFrame(data['records'])

# Nous récupérons les données de la colonne 'fields'
df_parking_relais_nettoye = pd.json_normalize(df_parking_relais['fields'])
df_parking_relais_nettoye = df_parking_relais_nettoye[['nom', 'nb_places', 'xlong', 'ylat', 'adresse','type_ouvrage', 'gratuit', 'nb_voitures', 'nb_velo', 'geo_point_2d']]

# Remplacer les valeurs de la colonne gratuit
df_parking_relais_nettoye['gratuit'] = df_parking_relais_nettoye['gratuit'].replace('F', 'non').replace('T', 'oui')
# Renommer la colonne geo_point_2d
df_parking_relais_nettoye = df_parking_relais_nettoye.rename(columns={'geo_point_2d':'lat&lon'})
# Passer la colonne lat&lon en string et retirer premier et dernier crochet de la colonne lat&lon
df_parking_relais_nettoye['lat&lon'] = df_parking_relais_nettoye['lat&lon'].astype(str).apply(lambda x : x[1:]).apply(lambda x : x[:-1])

# Ajout de la colonne "Relais"
df_parking_relais_nettoye['relais ?'] = 'oui'

# Rajouter une colonne qui précise le gestionnaire des parkings relais
df_parking_relais_nettoye['gestionnaire'] = 'Tisseo'

# Ensemble des parkings INDIGO, TM et RELAIS

In [ ]:
df_parking_global = pd.concat([df_parking_relais_nettoye, df_parking_indigo_nettoye], ignore_index=True)

# Afficher un DF avec les 5 parkings les plus proches de l'adresse renseignée + stocker les 5 parkings dans une liste pour faire une menu déroulant

In [ ]:
# Adresse renseignée par l'utilisateur
address_reference = input("D'où partez-vous ? \n")

# Liste des coordonnées à comparer
coordinates_to_compare = df_parking_global['lat&lon']

# Initialisation du géocodeur
geolocator = Nominatim(user_agent="my_app")

# Géocodage de l'adresse renseignée par l'utilisateur
location_reference = geolocator.geocode(address_reference)

# Vérification des coordonnées géographiques
if location_reference is not None:

    # Extraction des coordonnées de l'adresse de référence
    coordinates_reference = (location_reference.latitude, location_reference.longitude)

    # Initialisation du dictionnaire pour stocker les distances et les adresses
    distances = {}

    # Parcours des adresses à comparer
    for coordinate_to_compare in coordinates_to_compare:
        distance = geodesic(coordinates_reference, coordinate_to_compare).meters

        # Stockage de la distance et de l'adresse dans le dictionnaire
        distances[coordinate_to_compare] = distance

    # Tri du dictionnaire par distance et récupération des 5 Parkings les plus proches
    closest_parkings = sorted(distances, key=distances.get)[:5]

    if closest_parkings:
        # Création du DataFrame pour stocker les données des 5 parkings les plus proches
        df_5_parkings_proches = pd.DataFrame(columns=['Parkings', 'Type de parking', 'Gratuit', 'Nb_places_totales', 'Adresse', 'Distance_pour_y_acceder(m)', 'lat', 'lon'])

        # Parcours des parkings les plus proches
        for parking in closest_parkings:
            # Récupération des données du parking
            parking_data = df_parking_global[df_parking_global['lat&lon'] == parking]

            # Récupération des valeurs spécifiques
            nom = parking_data['nom'].values[0]
            type_parking = parking_data['type_ouvrage'].values[0]
            gratuit = parking_data['gratuit'].values[0]
            nb_places = parking_data['nb_voitures'].values[0]
            adresse = parking_data['adresse'].values[0]
            lat = parking_data['ylat'].values[0]
            lon = parking_data['xlong'].values[0]

            # Affichage de la distance
            distance = distances[parking]
            # print("{:.0f} mètres".format(distance))

            # Ajout du parking au DataFrame
            df_5_parkings_proches = df_5_parkings_proches.append({
                'Parkings': nom,
                'Type de parking': type_parking,
                'Gratuit': gratuit,
                'Nb_places_totales': nb_places,
                'Adresse': adresse,
                'lat' : lat,
                'lon' : lon,
                'Distance_pour_y_acceder(m)' : round(distance)

            }, ignore_index=True)


        # Stocker les 5 parkings les plus proches dans une liste pour créer une menu déroulant
        menu_deroulant_parkings = df_parking_global[df_parking_global['lat&lon'].isin(closest_parkings)]['nom'].tolist()
        #print("Parkings les plus proches :", nom_parking)

    else:
        print("Aucune adresse trouvée parmi la liste")

else:
    print("Adresse de référence introuvable")


# Nous supprimons les deux colonnes lat & lon 
df_5_parkings_proches_dash = df_5_parkings_proches.drop(columns=['lon', 'lat'], axis=1)

# Faire démarrer l'indexation à 1 au lieu de 0   
df_5_parkings_proches_dash = df_5_parkings_proches_dash.reset_index(drop=True)
df_5_parkings_proches_dash.index = df_5_parkings_proches_dash.index + 1

# Afficher le DF final pour l'utilisateur
df_5_parkings_proches_dash

# Test avec : Place Arnaud Bernard, 31000, Toulouse

,Parkings,Type de parking,Gratuit,Nb_places_totales,Adresse,Distance_pour_y_acceder(m)
1,ARNAUD BERNARD,Enterré,non,232,"Place Arnaud Bernard, 31000, Toulouse",35
2,JEANNE D'ARC,Enterré,non,370,"Place Jeanne d'Arc, 31000, Toulouse",636
3,EUROPE,Enterré,non,337,"Place de l'Europe, 31000, Toulouse",660
4,VICTOR HUGO,Aérien,non,385,"Place Victor Hugo, 31000, Toulouse",767
5,CAPITOLE,Enterré,non,748,"Place du Capitole, 31000, Toulouse",825


In [ ]:
# Créer un menu déroulant qui permet à l'utilisateur de selectionner le parking qu'il souhaite 
# Va nous servir à lui faire une proposition de station METRO, BUS et/ou TRAM & BORNE VELO, 
menu_deroulant_parkings

['EUROPE', "JEANNE D'ARC", 'VICTOR HUGO', 'CAPITOLE', 'ARNAUD BERNARD']

In [ ]:
# Aller récupérer l'adresse du parking choisi : A FAIRE

# Prenons pour exemple le parking Capitole
nom_du_parking_choisi = 'Capitole'
adresse_du_parking_choisi = df_5_parkings_proches_dash[df_5_parkings_proches_dash['adresse'].str(nom_du_parking_choisi)]

KeyError: 'adresse'

# Afficher une carte avec les 5 parkings en question 

In [ ]:
# Créer une carte folium avec les 5 parkings de notre DF df_5_parkings_proches
carte_5_parkings_les_plus_proches = folium.Map(location=[lat, lon], zoom_start=15)

for i in range(5):
    lat_parking = df_5_parkings_proches['lat'][i]
    lon_parking = df_5_parkings_proches['lon'][i]
    nom_parking = f'PARKING_{i + 1}'

    folium.Marker(location=[lat_parking, lon_parking], popup="<i>Nom du parking :\n</i>"+df_5_parkings_proches['Parkings'][i]).add_to(carte_5_parkings_les_plus_proches)

carte_5_parkings_les_plus_proches

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7c80b052-7ead-4799-b132-a25d57707958' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>